In [2]:
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster

In [3]:
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode("Champ de Mars, Paris, France")

In [4]:
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))


Latitude = 48.85614465, Longitude = 2.297820393322227


In [5]:
df = pd.read_csv("addresses.csv")
df.head()

,Unnamed: 0,Typ,Nr,Namn,Address1,Address3,Address4,Address5,Telefon
0,0,Butik,102,Fältöversten,Karlaplan 13,115 20,STOCKHOLM,Stockholms län,08/662 22 89
1,1,Butik,104,NaN,Nybrogatan 47,114 39,STOCKHOLM,Stockholms län,08/662 50 16
2,2,Butik,106,Garnisonen,Karlavägen 100 A,115 26,STOCKHOLM,Stockholms län,08/662 64 85
3,3,Butik,110,NaN,Hötorgshallen,111 57,STOCKHOLM,Stockholms län,08/56849241
4,4,Butik,113,Sergel,Drottninggatan 45,111 21,STOCKHOLM,Stockholms län,08/21 47 44


In [6]:
df ['ADDRESS'] = df['Address1'].astype(str) + ', ' + \
                 df ['Address3'] + ', ' + \
                 df ['Address4'] + ', ' + \
                 df ['Address5'] + ', ' + 'Sweden'
df.head()

,Unnamed: 0,Typ,Nr,Namn,Address1,Address3,Address4,Address5,Telefon,ADDRESS
0,0,Butik,102,Fältöversten,Karlaplan 13,115 20,STOCKHOLM,Stockholms län,08/662 22 89,"Karlaplan 13, 115 20, STOCKHOLM, Stockholms lä..."
1,1,Butik,104,NaN,Nybrogatan 47,114 39,STOCKHOLM,Stockholms län,08/662 50 16,"Nybrogatan 47, 114 39, STOCKHOLM, Stockholms l..."
2,2,Butik,106,Garnisonen,Karlavägen 100 A,115 26,STOCKHOLM,Stockholms län,08/662 64 85,"Karlavägen 100 A, 115 26, STOCKHOLM, Stockholm..."
3,3,Butik,110,NaN,Hötorgshallen,111 57,STOCKHOLM,Stockholms län,08/56849241,"Hötorgshallen, 111 57, STOCKHOLM, Stockholms l..."
4,4,Butik,113,Sergel,Drottninggatan 45,111 21,STOCKHOLM,Stockholms län,08/21 47 44,"Drottninggatan 45, 111 21, STOCKHOLM, Stockhol..."


In [7]:
from geopy.extra.rate_limiter import RateLimiter
# convenient function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds = 1)
# create location column
df ['location'] = df ['ADDRESS'].apply(geocode)
# create longitude, latitude, and altitude from location column (return type is tuple)
df ['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)


RateLimiter caught an error, retrying (0/2 tries). Called with (*('Odengatan 92, 113 22, STOCKHOLM, Stockholms län, Sweden',), **{}).
Traceback (most recent call last):
  File "/Users/abeerhalani/opt/anaconda3/lib/python3.7/urllib/request.py", line 1319, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Users/abeerhalani/opt/anaconda3/lib/python3.7/http/client.py", line 1252, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Users/abeerhalani/opt/anaconda3/lib/python3.7/http/client.py", line 1298, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Users/abeerhalani/opt/anaconda3/lib/python3.7/http/client.py", line 1247, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Users/abeerhalani/opt/anaconda3/lib/python3.7/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Users/abeerhalani/opt/anaconda3/lib/python3.7/http/client.py", line 9

In [8]:
df.head()

,Unnamed: 0,Typ,Nr,Namn,Address1,Address3,Address4,Address5,Telefon,ADDRESS,location,point
0,0,Butik,102,Fältöversten,Karlaplan 13,115 20,STOCKHOLM,Stockholms län,08/662 22 89,"Karlaplan 13, 115 20, STOCKHOLM, Stockholms lä...","(Karlaplan, Vedbäraren, Östermalm, Östermalms ...","(59.3383148, 18.0899599, 0.0)"
1,1,Butik,104,NaN,Nybrogatan 47,114 39,STOCKHOLM,Stockholms län,08/662 50 16,"Nybrogatan 47, 114 39, STOCKHOLM, Stockholms l...","(47, Nybrogatan, Brunfisken, Östermalm, Österm...","(59.3372072, 18.0790982, 0.0)"
2,2,Butik,106,Garnisonen,Karlavägen 100 A,115 26,STOCKHOLM,Stockholms län,08/662 64 85,"Karlavägen 100 A, 115 26, STOCKHOLM, Stockholm...","(Karlavägen, Infanteristen, Östermalm, Österma...","(59.3353799, 18.1006258, 0.0)"
3,3,Butik,110,NaN,Hötorgshallen,111 57,STOCKHOLM,Stockholms län,08/56849241,"Hötorgshallen, 111 57, STOCKHOLM, Stockholms l...","(Hötorgshallen, Hötorget, Konserthuset, Norrma...","(59.3343273, 18.0626039, 0.0)"
4,4,Butik,113,Sergel,Drottninggatan 45,111 21,STOCKHOLM,Stockholms län,08/21 47 44,"Drottninggatan 45, 111 21, STOCKHOLM, Stockhol...","(Drottninggatan, Torgvågen, Norrmalm, Norrmalm...","(59.3336068, 18.0616879, 0.0)"


In [9]:
df['point'][0][0]

59.3383148

In [10]:
# split point column into latitude, longitude, and altitude columns
df [['latitude','longitude','altitude']] = pd.DataFrame(df['point'].tolist(),index = df.index)
df.head()

,Unnamed: 0,Typ,Nr,Namn,Address1,Address3,Address4,Address5,Telefon,ADDRESS,location,point,latitude,longitude,altitude
0,0,Butik,102,Fältöversten,Karlaplan 13,115 20,STOCKHOLM,Stockholms län,08/662 22 89,"Karlaplan 13, 115 20, STOCKHOLM, Stockholms lä...","(Karlaplan, Vedbäraren, Östermalm, Östermalms ...","(59.3383148, 18.0899599, 0.0)",59.338315,18.089960,0.0
1,1,Butik,104,NaN,Nybrogatan 47,114 39,STOCKHOLM,Stockholms län,08/662 50 16,"Nybrogatan 47, 114 39, STOCKHOLM, Stockholms l...","(47, Nybrogatan, Brunfisken, Östermalm, Österm...","(59.3372072, 18.0790982, 0.0)",59.337207,18.079098,0.0
2,2,Butik,106,Garnisonen,Karlavägen 100 A,115 26,STOCKHOLM,Stockholms län,08/662 64 85,"Karlavägen 100 A, 115 26, STOCKHOLM, Stockholm...","(Karlavägen, Infanteristen, Östermalm, Österma...","(59.3353799, 18.1006258, 0.0)",59.335380,18.100626,0.0
3,3,Butik,110,NaN,Hötorgshallen,111 57,STOCKHOLM,Stockholms län,08/56849241,"Hötorgshallen, 111 57, STOCKHOLM, Stockholms l...","(Hötorgshallen, Hötorget, Konserthuset, Norrma...","(59.3343273, 18.0626039, 0.0)",59.334327,18.062604,0.0
4,4,Butik,113,Sergel,Drottninggatan 45,111 21,STOCKHOLM,Stockholms län,08/21 47 44,"Drottninggatan 45, 111 21, STOCKHOLM, Stockhol...","(Drottninggatan, Torgvågen, Norrmalm, Norrmalm...","(59.3336068, 18.0616879, 0.0)",59.333607,18.061688,0.0


In [11]:
df.columns

Index(['Unnamed: 0', 'Typ', 'Nr', 'Namn', 'Address1', 'Address3', 'Address4',
       'Address5', 'Telefon', 'ADDRESS', 'location', 'point', 'latitude',
       'longitude', 'altitude'],
      dtype='object')

In [12]:
df = df.drop(['Address1','Address3','Address4','Address5','Telefon','ADDRESS','location','point'], axis=1)

In [13]:
df.head()

,Unnamed: 0,Typ,Nr,Namn,latitude,longitude,altitude
0,0,Butik,102,Fältöversten,59.338315,18.089960,0.0
1,1,Butik,104,NaN,59.337207,18.079098,0.0
2,2,Butik,106,Garnisonen,59.335380,18.100626,0.0
3,3,Butik,110,NaN,59.334327,18.062604,0.0
4,4,Butik,113,Sergel,59.333607,18.061688,0.0


In [14]:
df.latitude.isnull().sum()

2

In [15]:
df = df[pd.notnull(df["latitude"])]

In [16]:
sweden_map =folium.Map(
    location=[59.338315,18.089960],
    tiles='OpenStreetMap',
    zoom_start=13,
)
def make_circle_marker(row):
    circle = folium.CircleMarker(location=[row["latitude"], row["longitude"]],
                                radius=5,
                                color="purple")
    circle.add_to(sweden_map)

In [19]:
df.apply(make_circle_marker, axis=1)
sweden_map.save(outfile='sweden_map.html')
sweden_map
